In [3]:
from proteins2info import InfoRetrieval

/Users/fabian/miniconda3/envs/auto-db-pipeline/lib/python3.8/site-packages/Bio/SubsMat/__init__.py:126: BiopythonDeprecationWarning: Bio.SubsMat has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.substitution_matrices as a replacement, and contact the Biopython developers if you still need the Bio.SubsMat module.
  warnings.warn(
Database path /Volumes/LaCie/sabdab-sabpred/data/ABDB was not found.


In [10]:
genbank = InfoRetrieval()
len(genbank.entries)

11493

In [11]:
genbank.filter_AB_entries()

Number of entires removed by antibody filter: 5166
Number of entires after antibody filter: 6327
----------


In [12]:
genbank.classify_vh_vl_anarci()

In [16]:
genbank.find_antigen({})

Number of entries where antigen was determined: 0
Number of entries where antigen was not determined: 6327
----------


In [17]:
genbank.find_fragment_id()

Number of entries where fragement name was determined: 1733
Number of entries where fragment name was not determined: 4594
----------


In [18]:
genbank.find_nanobodies()

In [19]:
genbank.seperate_nanobodies()

Number of nanobodies: 155
Number of antibodies: 6172
----------


In [20]:
genbank.group_by_publication()

In [21]:
genbank.pair_vh_vl()

Number of sequence pairs: 766
Number of sequences that could not be paired in attempt 1: 4640
----------


In [24]:
genbank.unpaired_entries[0][0]

{'GBSeq_locus': 'UGO91527',
 'GBSeq_moltype': 'AA',
 'GBSeq_update-date': '15-DEC-2021',
 'GBSeq_create-date': '15-DEC-2021',
 'GBSeq_definition': 'immunoglobulin heavy chain variable region, partial [Homo sapiens]',
 'GBSeq_accession-version': 'UGO91527.1',
 'GBSeq_source': 'Homo sapiens (human)',
 'GBSeq_sequence': 'qvqlqesgpglvkpsetlsltctvsggsissyywswirqppgkglewigyiyysgstnynpslksrvtisvdtsknqfslklssvtaadtavyycardirdssgwanwfdpwgqgtlvtvss',
 'GBSeq_references': [{'GBReference_authors': ['Cho,H.',
    'Gonzales-Wartz,K.K.',
    'Huang,D.',
    'Yuan,M.',
    'Peterson,M.',
    'Liang,J.',
    'Beutler,N.',
    'Torres,J.L.',
    'Cong,Y.',
    'Postnikova,E.',
    'Bangaru,S.',
    'Talana,C.A.',
    'Shi,W.',
    'Yang,E.S.',
    'Zhang,Y.',
    'Leung,K.',
    'Wang,L.',
    'Peng,L.',
    'Skinner,J.',
    'Li,S.',
    'Wu,N.C.',
    'Liu,H.',
    'Dacon,C.',
    'Moyer,T.',
    'Cohen,M.',
    'Zhao,M.',
    'Lee,F.E.',
    'Weinberg,R.S.',
    'Douagi,I.',
    'Gross,R.',
    'Schm

In [25]:
import pandas as pd
df = pd.read_csv('sabdab_summary_all.tsv', sep='\t')

In [26]:
df.head()

,pdb,Hchain,Lchain,model,antigen_chain,antigen_type,antigen_het_name,antigen_name,short_header,date,...,scfv,engineered,heavy_subclass,light_subclass,light_ctype,affinity,delta_g,affinity_method,temperature,pmid
0,7srr,E,e,0,B | C,protein | protein,NA | NA,g protein subunit q (gi2-mini-gq chimera) | gu...,MEMBRANE PROTEIN,09/21/22,...,True,True,unknown,unknown,unknown,None,None,None,None,None
1,7srs,Q,P,0,R,protein,NaN,5-hydroxytryptamine receptor 2b,MEMBRANE PROTEIN,09/21/22,...,False,True,IGHV1,IGKV4,Kappa,None,None,None,None,None
2,7srs,H,L,0,C,protein,NaN,isoform 1b of beta-arrestin-1,MEMBRANE PROTEIN,09/21/22,...,False,True,IGHV3,IGKV1,Kappa,None,None,None,None,None
3,7ut3,H,L,0,L,Hapten,TI0,4-nitrophenyl 2-acetamido-2-deoxy-alpha-D-gala...,IMMUNE SYSTEM,09/21/22,...,False,True,IGHV5,IGKV4,Kappa,None,None,None,None,None
4,7v5j,E,D,0,A,protein,NaN,spike glycoprotein,PROTEIN BINDING,09/21/22,...,False,True,IGHV1,IGKV1,Kappa,None,None,None,None,None


In [16]:
with open('../../src/covid_known_antigens.txt', 'r') as antigens_file:
    antigens_text = antigens_file.readlines()[1:]

In [12]:
antigens.split(': ')

['antigen',
 'possible names/abbreviations\nCOVID',
 'coronavirus, sars-cov, sars\nMERS',
 'mers-cov, mers\nSARS-COV-2',
 'sars-cov-2, covid-19\nSARS-COV-1',
 'sars-cov-1\nSpike protein',
 'spike, spike protein\nRBD',
 'receptor binding domain, rbd']

In [24]:
known_antigens = dict()
for line in antigens_text:
    antigen = line.strip().split(': ')[0]
    names = line.strip().split(': ')[1].split(', ')
    known_antigens[antigen] = names

In [25]:
known_antigens

{'COVID': ['coronavirus', 'sars-cov', 'sars'],
 'MERS': ['mers-cov', 'mers'],
 'SARS-COV-2': ['sars-cov-2', 'covid-19'],
 'SARS-COV-1': ['sars-cov-1'],
 'Spike protein': ['spike', 'spike protein'],
 'RBD': ['receptor binding domain', 'rbd']}